# Tagging (Sentiment Analysis)

## Intro and how to classify simple text

Let's begin with running a LLM to perform classification of a given piece of text. We can then use this perform classification and sentiment analysis for a bunch of different use cases. You will see some of these applications already in amazon review summaries for example.

In [ ]:
from langchain_ollama.llms import OllamaLLM

llm = OllamaLLM(model="llama3.1:8b")

In [ ]:
llm.invoke("Sentiment analysis: 'You are the best LLM'")

In [ ]:
llm.invoke("""Sentiment analysis in one word: 'Was so excited to get these especially after 
            reading all the reviews but when i opened them i noticed the rings were splintering. 
            I tried them once and used the velcro wrap around to secure the extra length of straps in place, and it tore off immediately'""")

In [ ]:
llm.invoke("Sentiment analysis in one word and return nothing else: 'Was so excited to get these especially after reading all the reviews but when i opened them i noticed the rings were splintering. I tried them once and used the velcro wrap around to secure the extra length of straps in place, and it tore off immediately'")

In [ ]:
llm.invoke("Sentiment analysis in one word positive, negative or neutral and return nothing else: 'Was so excited to get these especially after reading all the reviews but when i opened them i noticed the rings were splintering. I tried them once and used the velcro wrap around to secure the extra length of straps in place, and it tore off immediately'")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_ollama import ChatOllama

tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)


class Classification(BaseModel):
    sentiment: str = Field(description="The sentiment of the text")
    aggressiveness: int = Field(
        description="How aggressive the text is on a scale from 1 to 10"
    )
    language: str = Field(description="The language the text is written in")


llm = ChatOllama(model="llama3.1:8b").with_structured_output(
    Classification
)

tagging_chain = tagging_prompt | llm

In [ ]:
inp = """Was so excited to get these especially after reading all the reviews but when i opened them i noticed the rings were splintering. 
        I tried them once and used the velcro wrap around to secure the extra length of straps in place, and it tore off immediately"""
tagging_chain.invoke({"input": inp})